In [1]:
import sys
import math
import time
from pathlib import Path
from typing import Callable, Tuple

import torch
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(str(Path("..").resolve()))
from data_handling import load_measurements_npz, MeasurementDataset, MeasurementLoader
from hyper_rbm import SymmetricHyperRBM, train_loop, get_sigmoid_curve


data_dir = Path("measurements")
models_dir = Path("models")
models_dir.mkdir(parents=True, exist_ok=True)
results_dir = Path("results")
results_dir.mkdir(parents=True, exist_ok=True)

device = "cpu"
print(f"Running on: {device}")

Running on: cpu


In [2]:
def compute_mz_abs(samples: torch.Tensor) -> Tuple[float, float]:
    # convert spins into +1/-1 domain, take absolute mean per sample then batch
    spins_pm = 1.0 - 2.0 * samples.float()
    mz_per_sample = spins_pm.mean(dim=1)
    mz_abs = mz_per_sample.abs()

    total_mz = mz_abs.mean().item()
    total_mz_err = mz_abs.std(unbiased=True).item() / math.sqrt(samples.shape[0])
    return total_mz, total_mz_err

def compute_mx(samples: torch.Tensor, log_score_fn: Callable[[torch.Tensor], torch.Tensor]) -> Tuple[float, float]:

    B, N = samples.shape
    with torch.no_grad():
        log_scores_orig = log_score_fn(samples)
        total_ratios = torch.zeros(B, device=samples.device, dtype=torch.float32)

        # iterate over all sites in batch manner and calculate ratios in log space. accumulate all site ratios
        for i in range(N):
            flipped_samples = samples.clone()
            flipped_samples[:, i] = 1 - flipped_samples[:, i]
            log_scores_flip = log_score_fn(flipped_samples)
            log_ratios = log_scores_flip - log_scores_orig
            total_ratios += torch.exp(log_ratios)

        avg_mx_per_sample = total_ratios / N
        total_mx = avg_mx_per_sample.mean().item()
        total_mx_err = avg_mx_per_sample.std(unbiased=True).item() / math.sqrt(B)
        return total_mx, total_mx_err

In [3]:
SIDE_LENGTH = 4
FILE_SAMPLE_COUNT = 20_000
TRAIN_SAMPLE_COUNT = 20_000

h_support = [1.00, 2.10, 3.30, 7.00]
h_novel = [1.5, 1.8, 2.4, 2.6, 2.80, 2.9, 3.0, 3.1, 3.4, 3.6, 3.8, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5]
all_h_values = sorted(list(set(h_support + h_novel)))

file_names = [f"tfim_{SIDE_LENGTH}x{SIDE_LENGTH}_h{h:.2f}_{FILE_SAMPLE_COUNT}.npz" for h in h_support]
file_paths = [data_dir / fn for fn in file_names]

print(f"Support points: {h_support}")
print(f"Training on {TRAIN_SAMPLE_COUNT} samples per support point.")

samples_per_support = [TRAIN_SAMPLE_COUNT] * len(file_paths)
dataset = MeasurementDataset(file_paths, load_measurements_npz, ["h"], samples_per_support)

Support points: [1.0, 2.1, 3.3, 7.0]
Training on 20000 samples per support point.


In [4]:
N_EPOCHS = 50
BATCH_SIZE = 1024
NUM_HIDDEN = 64
HYPER_NET_WIDTH = 64
K_STEPS = 10
GIBBS_NOISE_FRAC = 0.1
INIT_LR = 1e-2
FINAL_LR = 1e-4

seed = 42
torch.manual_seed(seed)
rng = torch.Generator().manual_seed(seed)

loader = MeasurementLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False, rng=rng)

model = SymmetricHyperRBM(num_v=dataset.num_qubits, num_h=NUM_HIDDEN,
                          hyper_dim=HYPER_NET_WIDTH, k=K_STEPS).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=INIT_LR)
scheduler = get_sigmoid_curve(INIT_LR, FINAL_LR, N_EPOCHS*len(loader), 0.005)

model = train_loop(model, optimizer, loader, num_epochs=N_EPOCHS,
                   lr_schedule_fn=scheduler, noise_frac=GIBBS_NOISE_FRAC, rng=rng)

Epoch  | Loss       | LR        
-----------------------------------
1      | +0.0566     | 0.009999
10     | +0.0035     | 0.009974
20     | +0.0109     | 0.008799
30     | +0.0062     | 0.001312
40     | -0.0029     | 0.000127
50     | -0.0161     | 0.000101


In [5]:
ref_file = Path(f"tfim_{SIDE_LENGTH}x{SIDE_LENGTH}_magnet_ref.csv")
ref_df = pd.read_csv(ref_file)
print(f"Loaded ED reference data from: {ref_file.name}\n")

SAMPLES_EVAL = 10_000
K_STEPS_EVAL = 100
RNG_EVAL = torch.Generator(device=device).manual_seed(42)

sweep = sorted([(h, "support") for h in h_support] + [(h, "novel") for h in h_novel], key=lambda x: x[0])

print(f"Evaluating magnetizations over {len(sweep)} field values...")
results = []

# constant temperature schedule at T=1.0 during generation
schedule_tensor = torch.tensor([1.0] * K_STEPS_EVAL, device=device, dtype=torch.float32)

for h_val, kind in sweep:
    cond_batch = torch.full((SAMPLES_EVAL, 1), h_val, device=device, dtype=torch.float32)

    samples = model.generate(cond_batch, T_schedule=schedule_tensor, rng=RNG_EVAL)

    # scorer wrapper for Mx calculation
    scorer = lambda v: model.log_score(v, cond_batch)

    mz, _ = compute_mz_abs(samples)
    mx, _ = compute_mx(samples, scorer)

    results.append({"h": h_val, "type": kind, "mz": mz, "mx": mx})
    print(f"h {h_val:5.2f} [{kind:7}] | |Mz|: {mz:.5f} | Mx: {mx:.5f}")

magnet_df = pd.DataFrame(results).sort_values("h")

Loaded ED reference data from: tfim_4x4_magnet_ref.csv

Evaluating magnetizations over 21 field values...
h  1.00 [support] | |Mz|: 0.96770 | Mx: 0.25164
h  1.50 [novel  ] | |Mz|: 0.93244 | Mx: 0.35972
h  1.80 [novel  ] | |Mz|: 0.88999 | Mx: 0.44969
h  2.10 [support] | |Mz|: 0.82474 | Mx: 0.55473
h  2.40 [novel  ] | |Mz|: 0.73211 | Mx: 0.66618
h  2.60 [novel  ] | |Mz|: 0.64645 | Mx: 0.74753
h  2.80 [novel  ] | |Mz|: 0.56874 | Mx: 0.80768
h  2.90 [novel  ] | |Mz|: 0.53319 | Mx: 0.83516
h  3.00 [novel  ] | |Mz|: 0.49323 | Mx: 0.86135
h  3.10 [novel  ] | |Mz|: 0.46262 | Mx: 0.87862
h  3.30 [support] | |Mz|: 0.39886 | Mx: 0.91551
h  3.40 [novel  ] | |Mz|: 0.38245 | Mx: 0.92231
h  3.60 [novel  ] | |Mz|: 0.33770 | Mx: 0.94393
h  3.80 [novel  ] | |Mz|: 0.30835 | Mx: 0.95569
h  4.00 [novel  ] | |Mz|: 0.28872 | Mx: 0.96247
h  4.50 [novel  ] | |Mz|: 0.25846 | Mx: 0.97494
h  5.00 [novel  ] | |Mz|: 0.24956 | Mx: 0.97927
h  5.50 [novel  ] | |Mz|: 0.24389 | Mx: 0.98282
h  6.00 [novel  ] | |Mz|: 0.23

In [6]:
support_df = magnet_df[magnet_df["type"] == "support"]
novel_df = magnet_df[magnet_df["type"] == "novel"]

plt.figure(figsize=(8, 5), dpi=100)

plt.plot(ref_df["h"], ref_df["mag_z"], ':', color='gray', label="Ref")
plt.plot(ref_df["h"], ref_df["mag_x"], ':', color='gray')

plt.plot(support_df["h"], support_df["mz"], 'o', color='tab:blue', label="Mz")
plt.plot(novel_df["h"], novel_df["mz"], 'o', color='tab:blue', mfc='none')

plt.plot(support_df["h"], support_df["mx"], 'D', color='tab:orange', label="Mx")
plt.plot(novel_df["h"], novel_df["mx"], 'D', color='tab:orange', mfc='none')

plt.xlabel("Transverse Field")
plt.ylabel("Magnetization")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [7]:
timestamp = time.strftime("%Y%m%d_%H%M%S")
file_name = f"tfim_{SIDE_LENGTH}x{SIDE_LENGTH}_magnet_two_rbm_{timestamp}.csv"
save_path = results_dir / file_name

magnet_df.to_csv(save_path, index=False, float_format="%.6f")
print(f"Saved RBM magnetizations to: {save_path}")

Saved RBM magnetizations to: results/tfim_4x4_magnet_two_rbm_20260119_193410.csv
